In [85]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers,models
import pandas as pd
import os

On importe notre jeu de donnée puis on remplace les valeurs textuelle par des entiers car cela est plus facile à traiter

In [45]:
data = pd.read_csv("train.csv")
data['Class'].replace(['YOUNG','MIDDLE','OLD'],[0,1,2],inplace=True)#inplace true pour juste modifier le dataframe et pas en créer un nouveau

In [46]:
print('Regardons le format de ces donnees\n')
print(data.head(5))

Regardons le format de ces donnees

          ID  Class
0    377.jpg      1
1  17814.jpg      0
2  21283.jpg      1
3  16496.jpg      0
4   4487.jpg      1


In [47]:
def lireImage(chemin):
	img = tf.io.read_file(chemin)
	img = tf.image.decode_jpeg(img, channels=3) # channel 3 pour une image en couleur
	img = tf.image.convert_image_dtype(img, dtype=tf.float32) # dtype le format de donnée
	img = tf.image.resize(img, (150,150)) # 150 de largeur 150 de longueur
	return img

In [48]:
def charger(chemin, reponse):
    img = lireImage(chemin)
    return (img, reponse)

In [49]:
listeChemin = []
listeReponse = []
for chemin in os.listdir("Train"):
    listeChemin.append("Train"+"/"+chemin)
for i in listeChemin:
    tete,queue = os.path.split(i) #Sépare le path dans un tuple tete queue où la queue represente les donnees
    reponse = data.loc[data['ID'] == queue]['Class'].values[0] #loc permet de rechercher selon
    #une expression booléene donc ici on recherche quand l'ID est égal a la queue à la case Class de notre dataframe et on récupére la valeur 
    listeReponse.append(reponse)

In [13]:
tailleEntrainement = int(0.95*(len(listeChemin)))
tailleTest = int(0.05*(len(listeChemin)))

entrainement = tf.data.Dataset.from_tensor_slices((listeChemin[:tailleEntrainement], listeReponse[:tailleEntrainement]))
test = tf.data.Dataset.from_tensor_slices((listeChemin[tailleTest:], listeReponse[tailleTest:]))

entrainement = (entrainement
    .map(charger, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(64)
    .prefetch(tf.data.AUTOTUNE)
)
test = (test
    .map(charger, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(64)
    .prefetch(tf.data.AUTOTUNE)
)

In [14]:
model = models.Sequential([
    layers.Conv2D(filters=30, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3), padding = 'same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [26]:
model.fit(entrainement, epochs=10, validation_data=test)

Epoch 1/10
296/296 [==============================] - 312s 1s/step - loss: 0.3773 - accuracy: 0.8509 - val_loss: 0.4119 - val_accuracy: 0.8331
Epoch 2/10
296/296 [==============================] - 310s 1s/step - loss: 0.3077 - accuracy: 0.8796 - val_loss: 0.3976 - val_accuracy: 0.8433
Epoch 3/10
296/296 [==============================] - 309s 1s/step - loss: 0.2574 - accuracy: 0.8995 - val_loss: 0.3347 - val_accuracy: 0.8700
Epoch 4/10
296/296 [==============================] - 296s 1s/step - loss: 0.2106 - accuracy: 0.9207 - val_loss: 0.2728 - val_accuracy: 0.9008
Epoch 5/10
296/296 [==============================] - 291s 983ms/step - loss: 0.1814 - accuracy: 0.9311 - val_loss: 0.2771 - val_accuracy: 0.8982
Epoch 6/10
296/296 [==============================] - 277s 936ms/step - loss: 0.1488 - accuracy: 0.9448 - val_loss: 0.2913 - val_accuracy: 0.8993
Epoch 7/10
296/296 [==============================] - 269s 909ms/step - loss: 0.1345 - accuracy: 0.9501 - val_loss: 0.3245 - val_accurac

In [29]:
testPred = model.predict(test)
resultat = [np.argmax(item) for item in testPred]
print("Reponse données par le modèle ",resultat[:20])

reponseAttendu = listeReponse[tailleTest:]
print("Reponse attendu", reponseAttendu[:20])

296/296 [==============================] - 66s 224ms/step
Reponse données par le modèle  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]
Reponse attendu [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]


In [90]:
testImagePerso =  tf.data.Dataset.from_tensor_slices((['Test/moi.png'],[0]))
testImagePerso = (testImagePerso
    .map(charger, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(64)
    .prefetch(tf.data.AUTOTUNE)          
    )
testPredPerso = model.predict(testImagePerso)
resultatTest = [np.argmax(item) for item in testPredPerso]
print("reponse donnée par le modèle ",resultatTest)

1/1 [==============================] - 0s 103ms/step
reponse donnée par le modèle  [0]
